In [8]:
import collections, datetime
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import imdb2sheet

In [9]:
movies = imdb2sheet.load_movies_extra()
df = pd.DataFrame(movies)

len(movies)

1847

In [10]:
df.iloc[0]

id                                                           tt1001526
title                                                         Megamind
type                                                   MovieType.MOVIE
genres               [Animation, Action, Comedy, Crime, Family, Rom...
directors                                                [Tom McGrath]
year                                                              2010
release_date                                                2010-10-28
runtime                                                0 days 01:35:00
num_ratings                                                     245928
imdb_rating                                                        7.3
my_rating                                                            5
date_rated                                                  2013-04-15
plot                 Evil genius Megamind finally defeats his do-go...
writers                               [Alan Schoolcraft, Brent Simons]
actors

In [ ]:
# Compute diff my-rating <> imdb-rating and replot the graphs (actors,...)

# Display table movies sorted by difference imdb <> my rating

# Display

# Add filter per genre (dash)

In [22]:
total_watched_time = sum([m.runtime for m in movies], datetime.timedelta())

remainder = total_watched_time.total_seconds()
months, _           = divmod(remainder, 30*24*60*60)
weeks, remainder    = divmod(remainder,  7*24*60*60)
days, remainder     = divmod(remainder,    24*60*60)
hours, remainder    = divmod(remainder,       60*60)
minutes, remainder  = divmod(remainder,          60)
assert remainder == 0.0
print(f'{weeks=} (~{weeks * 7 / 30:.1f} months), {days=}, {hours=}, {minutes=}')

weeks=19.0 (~4.4 months), days=1.0, hours=23.0, minutes=55.0


In [12]:
movies_by_runtime = sorted(movies, key=lambda m: m.runtime, reverse=True)

pd.DataFrame(movies_by_runtime[:20])

,id,title,type,genres,directors,year,release_date,runtime,num_ratings,imdb_rating,...,writers,actors,languages,countries,pg_rating,poster,awards,box_office,rotten_rating,metacritic_rating
0,tt0499410,That Mitchell and Webb Look,MovieType.TV_SERIES,"[Comedy, Fantasy]",[],2006,2006-09-14,0 days 12:00:00,8731,8.0,...,[N/A],"[David Mitchell, Robert Webb, James Bachman]",[English],[United Kingdom],N/A,https://m.media-amazon.com/images/M/MV5BMTI4OT...,Won 1 BAFTA Film Award1 win & 8 nominations total,0,-1,-1
1,tt10048342,The Queen's Gambit,MovieType.TV_MINI_SERIES,[Drama],[],2020,2020-10-23,0 days 06:35:00,375737,8.6,...,"[Scott Frank, Allan Scott]","[Anya Taylor-Joy, Chloe Pirrie, Bill Camp]","[English, French, Spanish, Russian]",[United States],TV-MA,https://m.media-amazon.com/images/M/MV5BM2EwMm...,Won 11 Primetime Emmys. 51 wins & 53 nominatio...,0,-1,-1
2,tt0108167,Smoking/No Smoking,MovieType.MOVIE,"[Comedy, Drama]",[Alain Resnais],1993,1993-12-15,0 days 04:58:00,1786,7.4,...,"[Alan Ayckbourn, Jean-Pierre Bacri, Anne Dutter]","[Sabine Azéma, Pierre Arditi, Peter Hudson]",[French],"[France, Italy, Switzerland]",N/A,https://m.media-amazon.com/images/M/MV5BNWM0Mj...,11 wins & 8 nominations,57033,-1,-1
3,tt5491994,Planet Earth II,MovieType.TV_MINI_SERIES,[Documentary],[],2016,2016-11-06,0 days 04:58:00,107464,9.5,...,[N/A],"[David Attenborough, Gordon Buchanan, Chadden ...","[English, French]","[United Kingdom, Germany, France, China, Unite...",TV-G,https://m.media-amazon.com/images/M/MV5BZWYxOD...,Won 2 Primetime Emmys. 15 wins & 28 nomination...,0,-1,-1
4,tt0031381,Gone with the Wind,MovieType.MOVIE,"[Drama, History, Romance, War]","[Sam Wood, George Cukor, Victor Fleming]",1939,1939-12-15,0 days 03:58:00,304766,8.1,...,"[Margaret Mitchell, Sidney Howard, Oliver H.P....","[Clark Gable, Vivien Leigh, Thomas Mitchell]",[English],[United States],Passed,https://m.media-amazon.com/images/M/MV5BYjUyZW...,Won 10 Oscars. 21 wins & 12 nominations total,200852579,90,97
5,tt10133370,Stalk,MovieType.TV_SERIES,"[Drama, Thriller]",[],2019,2019-12-31,0 days 03:50:00,226,7.0,...,"[Simon Bouisson, Victor Rodenbach]","[Théo Fernandez, Carmen Kassovitz, Pablo Cobo]",[French],[France],N/A,https://m.media-amazon.com/images/M/MV5BYmU1OT...,4 wins,0,-1,-1
6,tt0052618,Ben-Hur,MovieType.MOVIE,"[Adventure, Drama, History]",[William Wyler],1959,1959-11-18,0 days 03:32:00,231223,8.1,...,"[Lew Wallace, Karl Tunberg, Gore Vidal]","[Charlton Heston, Jack Hawkins, Stephen Boyd]",[English],[United States],G,https://m.media-amazon.com/images/M/MV5BNjgxY2...,Won 11 Oscars. 29 wins & 16 nominations total,74422622,86,90
7,tt0049934,War and Peace,MovieType.MOVIE,"[Drama, Romance, War]",[King Vidor],1956,1956-08-21,0 days 03:28:00,9778,6.8,...,"[Lev Tolstoy, Bridget Boland, Robert Westerby]","[Audrey Hepburn, Henry Fonda, Mel Ferrer]","[English, Italian, Russian]","[United States, Italy]",PG,https://m.media-amazon.com/images/M/MV5BNmJjOT...,Nominated for 3 Oscars. 5 wins & 13 nomination...,0,44,-1
8,tt0167260,The Lord of the Rings: The Return of the King,MovieType.MOVIE,"[Action, Adventure, Drama, Fantasy]",[Peter Jackson],2003,2003-12-01,0 days 03:21:00,1737407,8.9,...,"[J.R.R. Tolkien, Fran Walsh, Philippa Boyens]","[Elijah Wood, Viggo Mortensen, Ian McKellen]","[English, Quenya, Old English, Sindarin]","[New Zealand, United States]",PG-13,https://m.media-amazon.com/images/M/MV5BNzA5ZD...,Won 11 Oscars. 209 wins & 124 nominations total,377845905,93,94
9,tt0054331,Spartacus,MovieType.MOVIE,"[Adventure, Biography, Drama, History, War]",[Stanley Kubrick],1960,1960-10-06,0 days 03:17:00,131289,7.9,...,"[Dalton Trumbo, Howard Fast, Peter Ustinov]","[Kirk Douglas, Laurence Olivier, Jean Simmons]",[English],[United States],PG-13,https://m.media-amazon.com/images/M/MV5BMjc4MT...,Won 4 Oscars. 12 wins & 11 nominations total,1830650,93,87


In [23]:
import dataclasses
movies_with_runtime = [dataclasses.replace(m, runtime=m.runtime.total_seconds() // 60) for m in movies]

df2 = pd.DataFrame(movies_with_runtime)
px.histogram(df2, x='runtime', color='type')

In [24]:
px.histogram(df, x='my_rating', color='type', range_x=[1, 10])

In [25]:
px.histogram(df, x='imdb_rating', color='type', range_x=[1, 10])

In [26]:
px.histogram(df, x='year', color='type')

In [27]:
px.histogram(df, x='date_rated', color='type')

In [28]:
px.histogram(df, x='type')

In [29]:
genre2movie = collections.defaultdict(list)

for m in movies:
    for g in m.genres:
        genre2movie[g].append(m)

vals = [
    {
        'genre': g, 
        'nb': len(v), 
        'avg_rating': np.average([f.my_rating for f in v]), 
        'titles': ', '.join([f.title for f in v[:3]]),
    }
    for g, v in genre2movie.items()
]
vals = sorted(vals, key=lambda v: v['nb'], reverse=True)
df2 = pd.DataFrame(vals)


px.bar(df2, x='genre', y='nb', hover_data=['titles', 'avg_rating'])

In [30]:

vals = sorted(vals, key=lambda v: v['avg_rating'], reverse=True)
df2 = pd.DataFrame(vals)
px.scatter(df2, x='genre', y='avg_rating', hover_data=['titles', 'nb'])

In [ ]:
px.scatter(df2, x='genre', y='avg_imdb_rating', hover_data=['titles', 'nb'])

In [28]:

df2 = pd.DataFrame(vals)
px.strip(df2, x='nb', y='avg_rating', hover_name='genre', hover_data=['titles'], log_x=True)

In [31]:
SIZE=1000

fig = px.strip(
    df, 
    x='my_rating', 
    y='imdb_rating', 
    hover_name='title',
    color='type',
    range_x=[1, 10],
    range_y=[1, 10],
    height=SIZE,
    width=SIZE,
)
fig.add_trace(go.Scatter(x=[1, 10], y=[1, 10], name='Identity'))
fig.update_xaxes(
    scaleanchor = "y",
    scaleratio = 1,
    gridwidth=0.3,
    tickvals=list(range(11)),
)
fig.update_yaxes(
    gridwidth=0.3,
    tickvals=list(range(11)),
)

# Improvements:
# * Regression lineaire

fig

In [32]:
SIZE=1000

fig = px.strip(
    df, 
    y='my_rating', 
    x='num_ratings', 
    hover_name='title',
    hover_data=['imdb_rating'],
    log_x=True,
    # color='imdb_rating'
    color='type',
)
fig

In [33]:
SIZE=1000


director2movies = collections.defaultdict(list)
for m in movies:
    for director in m.directors:
        director2movies[director].append(m)

vals = [
    {
        'director': director, 
        'nb': len(v), 
        'avg_rating': np.average([f.my_rating for f in v]), 
        'titles': ', '.join([f.title for f in v[:3]]),
    }
    for director, v in director2movies.items()
]
df2 = pd.DataFrame(vals)

fig = px.strip(
    df2, 
    y='avg_rating', 
    x='nb', 
    hover_name='director',
    hover_data=['titles'],
    log_x=True,
    # color='type',
    # range_x=[1, 10],
    # range_y=[1, 10],
    # height=SIZE,
    # width=SIZE,
)
fig

In [34]:
SIZE=1000


director2movies = collections.defaultdict(list)
for m in movies:
    for director in m.actors:
        director2movies[director].append(m)

vals = [
    {
        'actor': actor, 
        'nb': len(v), 
        'avg_rating': np.average([f.my_rating for f in v]), 
        'titles': ', '.join([f.title for f in v[:3]]),
    }
    for actor, v in director2movies.items()
]
df2 = pd.DataFrame(vals)

fig = px.strip(
    df2, 
    y='avg_rating', 
    x='nb', 
    hover_name='actor',
    hover_data=['titles'],
    # color='type',
    # range_x=[1, 10],
    # range_y=[1, 10],
    # height=SIZE,
    # width=SIZE,
)
# Color by sex
fig

In [35]:
import country_converter as coco
from etils import epy

cc = coco.CountryConverter()

def lookup(name):
    country2code = {
        'West Germany': 'DEU',
        'UK': 'GBR',
    }
    if name in country2code:
        return country2code[name]
    with epy.maybe_reraise(name):
        out = cc.convert(names=name, to='ISO3')
        if out == 'not found':
            raise ValueError(out)
        return out


countries2movies = collections.defaultdict(list)
for m in movies:
    for country in m.countries:
        country = lookup(country)
        countries2movies[country].append(m)

vals = [
    {
        'country': country, 
        'nb': len(v),
        'titles': ', '.join([f.title for f in v[:3]]),
    }
    for country, v in countries2movies.items()
]
df2 = pd.DataFrame(vals)


tick_text = [1, 5, 10, 50, 100, 500, 1000]
tick_vals = [np.log(t) for t in tick_text]


# How to have color log scale ?
fig = go.Figure()
fig.add_choropleth(
    locations=df2['country'],
    z=np.log(df2['nb']),
    text=df2['nb'],
    colorbar=dict(
        tickvals=tick_vals,
        ticktext=[str(t) for t in tick_text],
    )
)
fig

ModuleNotFoundError: No module named 'country_converter'

In [43]:
import pycountry
import tqdm



def lookup(name):
    country2code = {
        'Mandarin': 'CHN',
        'Shanghainese': 'CHN',
        'Cantonese': 'CHN',
        'Greek': 'GRC',
        'Gaelic': 'GBR',
        'Sotho': 'ZAF',
        'Punjabi': 'IND',
        'British Sign ': 'GBR',
        'Swahili': 'RWA',
        'Malay': 'SGP',
        'Sign ': 'GBR',
        'American Sign ': 'USA',
        'CHN': 'CHN',
        'GBR': 'GBR',
        'GRC': 'GRC',
        'Greenlandic': 'GRL',
        'Hokkien': 'CHN',
        'IND': 'IND',
        'Irish Gaelic': 'IRL',
        'Nama': 'NAM',
        'Old English': 'GBR',
        'Pashtu': 'AFG',
        'RWA': 'RWA',
        'SGP': 'SGP',
        'ZAF': 'ZAF',
        'North American Indian': 'USA',
        'N/A': '',
        'None': '',
    }
    if name in country2code:
        return country2code[name]
    out = pycountry.languages.get(name=name)
    if not out:
        raise ValueError(name)
    return out.alpha_3.upper()


countries2movies = collections.defaultdict(list)
for m in movies:
    for country in m.languages:
        country = lookup(country)
        countries2movies[country].append(m)

vals = [
    {
        'language': language, 
        'nb': len(v),
        'titles': ', '.join([f.title for f in v[:3]]),
    }
    for language, v in countries2movies.items()
]
vals = sorted(vals, key=lambda x: x['nb'], reverse=True)
df2 = pd.DataFrame(vals)

px.bar(df2, x='language', y='nb')


ValueError: Songhay